In [1]:
import pandas as pd
import os
import glob

In [36]:
# Import CSV with Subject list

# Set path variables. 
datadir = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/cest/7T_data"

# Import CSV with Subject list
input_csv_path = "~/Desktop/ImageData/PMACS_remote/data/subject_list_031124.csv"
df = pd.read_csv(input_csv_path)
df = df.dropna(subset=['BBLID','SCANID_CEST'])

# Create output df:
output_df = pd.DataFrame(columns=['BBLID', 'Session', 'Data_There'])


In [37]:
# Loop through rows of subjlist
for index, row in df.iterrows():
    bblid = row['BBLID']
    bblid = str(int(bblid))
    session = row['SCANID_CEST']
    session = str(int(session))
    
     # Check if subj data exists
    path = os.path.join(datadir, str(bblid), str(session))
    #print(path)
    data_there = "Yes" if os.path.isdir(path) else "No"
    #print(data_there)
    
    # Add a new row to the output DataFrame
    vals = [bblid, session, data_there]
    #output_df = output_df.append({'BBLID': bblid, 'Session': session, 'Data_There': data_there}, ignore_index=True)
    output_df.loc[len(output_df)] = vals

# Save the output DataFrame to a CSV file
output_csv_path = "output.csv"
output_df.to_csv(output_csv_path, index=False)



In [39]:
missing_cest_071924 = output_df[output_df['Data_There'] == 'No']
print(missing_cest_071924)
print(output_df)

    BBLID Session Data_There
25  96659   11096         No
87  96659   11096         No
88  90281   11093         No
      BBLID Session Data_There
0     20303   12234        Yes
1     90217   12230        Yes
2     88608   12108        Yes
3     21874   12094        Yes
4     94288   12092        Yes
..      ...     ...        ...
115   90877   10907        Yes
116   92155   11022        Yes
117   93274   10765        Yes
118  112126   11157        Yes
119  139272   10739        Yes

[120 rows x 3 columns]


In [ ]:
# Bash code to transfer over people that were hiding in old dataset.

bblid = ____
session = ___
# Source 
start=/project/bbl_projects/MotorGluCEST/dicoms

# End point 
finish=/project/bbl_roalf_pecsokphd/projects/glucest-rsfmri/data/cest/7T_data

mkdir $finish/$bblid

cp -r $start/$bblid/$session $finish/$bblid/$session

# Now, Loop through old dataset and see what extra data you have.

In [45]:
old_path = "~/Desktop/ImageData/PMACS_remote/data/olddata_summary.csv"
new_path = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/cest/7T_data"
df_old = pd.read_csv(old_path)
# Remove rows where Directory is the same as Subdirectory
df_old = df_old[df_old['Directory'] != df_old['Subdirectory']]

old_data_status_df = pd.DataFrame(columns=['BBLID', 'Session', 'Duplicated'])


# Loop through df_old and check for duplication
for index, row in df_old.iterrows():
    directory = row['Directory']
    #print(directory)
    subdirectory = row['Subdirectory']
    #print(subdirectory)
    path = os.path.join(new_path, str(directory), str(subdirectory))
    #print(path)
    duplicate = "Yes" if os.path.isdir(path) else "No"
    #print(str(directory), duplicate)
    vals = [directory, subdirectory, duplicate]
    #print(vals)
    old_data_status_df.loc[len(old_data_status_df)] = vals
#print(old_data_status_df)

     BBLID  Session Duplicated
0   100522    12003        Yes
1   102041    12037         No
2   111720    11766        Yes
3   112061    12261         No
4   112807    11890        Yes
5   114738    11706        Yes
6   114738    12095        Yes
7   115783    11788        Yes
8   115783    12179         No
9   116354    11774        Yes
10  116354    12180         No
11  118864    11783        Yes
12  118864    12144         No
13  121085    11716        Yes
14  121085    11970        Yes
15  125073    11814        Yes
16  125073    12167        Yes
17  127935    12101        Yes
18  128079    11934        Yes
19  130438    11999        Yes
20  131384    12198        Yes
21  132869    12109        Yes
22  135085    11812        Yes
23   19790    12171         No
24   20082    11821        Yes
25   20082    12183        Yes
26   20303    12234        Yes
27   20325    11852        Yes
28   20523    12186         No
29   20754    12200        Yes
30   20792    11887        Yes
31   207

In [52]:
# Now, Save the extra data in a csv
extra_data = old_data_status_df[old_data_status_df['Duplicated'] == 'No']
print(extra_data)
extra_data = extra_data.drop(columns=['Duplicated']) # This avoid annoying bash problem in next chunk
print(extra_data)
extra_data.to_csv("/Users/pecsok/Desktop/ImageData/PMACS_remote/data/cest/extra_data.csv", index=False)



     BBLID  Session Duplicated
1   102041    12037         No
3   112061    12261         No
8   115783    12179         No
10  116354    12180         No
12  118864    12144         No
23   19790    12171         No
28   20523    12186         No
36   22192    12036         No
37   22475    12096         No
38   22668    12257         No
39   22716    12266         No
40   23166    12246         No
43   89279    11699         No
44   89279    12078         No
57   96659    12256         No
     BBLID  Session
1   102041    12037
3   112061    12261
8   115783    12179
10  116354    12180
12  118864    12144
23   19790    12171
28   20523    12186
36   22192    12036
37   22475    12096
38   22668    12257
39   22716    12266
40   23166    12246
43   89279    11699
44   89279    12078
57   96659    12256


In [47]:
# Now, transfer over data from old data to a new folder in "data"

# Define the paths
old_path="/project/bbl_projects/MotorGluCEST/dicoms"
new_path="/project/bbl_roalf_pecsokphd/projects/glucest-rsfmri/data/cest/extra_data"
csv_file="extra_data.csv"

# Create the destination directory
mkdir -p "$new_path"

# Read the CSV file and process each row
while IFS=, read -r bblid session
do
    # Skip the header line
    if [[ "$bblid" == "BBLID" ]]; then
        continue
    fi
    
    # Construct source and destination paths
    src_dir="$old_path/$bblid/$session"
    dest_dir="$new_path/$bblid/$session"
    
    # Create destination directory if it doesn't exist
    mkdir -p "$(dirname "$dest_dir")"
    
    # Copy the directory
    cp -r "$src_dir" "$dest_dir"
    
    echo "Copied $src_dir to $dest_dir"
done < "$csv_file"



SyntaxError: invalid syntax (6112879.py, line 16)